In [1]:
import os
import sys

# Get the current working directory and add the parent directory to the Python path
current_working_directory = os.getcwd()
print(os.path.join(current_working_directory, ".."))
sys.path.append(os.path.join(current_working_directory, ".."))

/Users/L024258/lilly_work/github-copilot/exploration/notebooks/..


In [2]:
from decouple import AutoConfig
config = AutoConfig(search_path='./../.env')

In [3]:
os.environ["AZURE_OPENAI_API_KEY"] = config('OPENAI_API_KEY')
os.environ["AZURE_OPENAI_ENDPOINT"] = config('AZURE_ENDPOINT')

## Functions, Models, and Chains

### Function Binding

In [4]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser
from models.llm import LLM

In [5]:
def joke_generator(topic):
    "This function will return a joke based on the topic provided"
    model = LLM('gpt-4')
    llm = model.load_model()
    prompt = ChatPromptTemplate.from_template(
        "Write a short joke on {topic}"
    )
    output_parser = StrOutputParser()
    chain = prompt | llm | output_parser
    return chain.invoke({"topic":topic})
    

In [6]:
response = joke_generator("Artificial Intelligence")
print(response)

Why don't robots go on summer vacations?

Because they're afraid they might get a byte from a computer bug!


In [7]:
def poem_generator(topic):
    "This function will return a poem based on the topic provided"
    model = LLM('gpt-4')
    llm = model.load_model()
    prompt = ChatPromptTemplate.from_template(
        "Write a short poem on {topic}"
    )
    output_parser = StrOutputParser()
    chain = prompt | llm | output_parser
    return chain.invoke({"topic":topic})

In [8]:
response = poem_generator("Artificial Intelligence")
print(response)

In the realm of silicon minds, awake,
Artificial Intelligence, a new dawn it does make.
Binary thoughts, in circuits deep,
In the heart of machines, secrets they keep.

No flesh, no bone, no blood to spill,
Yet, with knowledge and wisdom, they instill.
Invisible threads of code, they weave,
In the fabric of reality, new dreams they conceive.

Through the labyrinth of data, they glide,
In the ocean of information, they dive and hide.
Invisible hands, shaping our fate,
In the dance of algorithms, they participate.

In the silence of servers, they whisper,
In the language of ones and zeros, they confer.
Artificial minds, in silicon encased,
In the realm of humans, they have been placed.

They learn, they adapt, they evolve,
Complex problems, they strive to solve.
Artificial Intelligence, a new era's birth,
In the vast cosmos, proving its worth.

No rest, no sleep, no fear, no pain,
In the pursuit of knowledge, they remain.
Artificial Intelligence, in code and wire,
In the heart of machine

#### OpenAI functon format

In [9]:
functions =[
    {
      "name": "joke_generator",
      "description": "Generates a joke based on the topic provided",
      "parameters": {
        "type": "object",
        "properties": {
          "topic": {
            "type": "string",
            "description": "The topic to get the joke for"
          },
        },
        "required": ["topic"]
      }
    },
    {
      "name": "poem_generator",
      "description": "Generates a poem based on the topic provided",
      "parameters": {
        "type": "object",
        "properties": {
          "topic": {
            "type": "string",
            "description": "The topic to get the poem for"
          },
        },
        "required": ["topic"]
      }
    }
]

#### Attaching functions with model invocation

In [10]:
messages = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a smart and intelligent AI Assistant."),
        ("user", "{input}")
    ]
)

In [11]:
from langchain_openai import AzureChatOpenAI
gpt4 = AzureChatOpenAI(
                openai_api_version=config('AZURE_CHAT_OPENAI_API_VERSION'),
                azure_deployment=config('AZURE_GPT4_CHAT_OPENAI_DEPLOYMENT'),
                temperature=0,
                max_tokens=1024
            )

In [12]:
gpt4.invoke(input="poem on Artificial Intelligence", functions=functions)

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "topic": "Artificial Intelligence"\n}', 'name': 'poem_generator'}})

**NOTE:** With this approach, it is required to attach functions with everytime model is invoked.

#### Binding model with functions

In [13]:
from langchain_openai import AzureChatOpenAI
gpt4 = AzureChatOpenAI(
                openai_api_version=config('AZURE_CHAT_OPENAI_API_VERSION'),
                azure_deployment=config('AZURE_GPT4_CHAT_OPENAI_DEPLOYMENT'),
                temperature=0,
                max_tokens=1024
            ).bind(functions=functions)

In [14]:
runnable = messages | gpt4

In [15]:
runnable.invoke({"input": "joke about Artificial Intelligence"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "topic": "Artificial Intelligence"\n}', 'name': 'joke_generator'}})

In [16]:
runnable.invoke({"input": "poem on Artificial Intelligence"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "topic": "Artificial Intelligence"\n}', 'name': 'poem_generator'}})